<a href="https://colab.research.google.com/github/FatemaSamir/pyspark/blob/main/NLP_Spam__Filter_Using_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Using PySpark

## Auther : Fatema Samir
## Date : 19/7/2022

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 57.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9378784855590e09f8f2e883b74ec438fabdc0411bb365908dcd87d8b2c2458a
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession 
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName('NLP_Project').getOrCreate() # intery point

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [ ]:
df = spark.read.option('sep','\t').text('/content/SMSSpamCollection.txt')
df.show()

+--------------------+
|               value|
+--------------------+
|ham\tGo until jur...|
|ham\tOk lar... Jo...|
|spam\tFree entry ...|
|ham\tU dun say so...|
|ham\tNah I don't ...|
|spam\tFreeMsg Hey...|
|ham\tEven my brot...|
|ham\tAs per your ...|
|spam\tWINNER!! As...|
|spam\tHad your mo...|
|ham\tI'm gonna be...|
|spam\tSIX chances...|
|spam\tURGENT! You...|
|ham\tI've been se...|
|ham\tI HAVE A DAT...|
|spam\tXXXMobileMo...|
|ham\tOh k...i'm w...|
|ham\tEh u remembe...|
|ham\tFine if that...|
|spam\tEngland v M...|
+--------------------+
only showing top 20 rows



In [ ]:
df = spark.read.option('sep','\t').csv('/content/SMSSpamCollection.txt',inferSchema=True)
df.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



### Print the schema

In [ ]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [ ]:
df_1 = df.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')
df_1.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [ ]:
df_1.show(10)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



In [ ]:
df_1.show(10,truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [ ]:
df_2 = df_1.withColumn('length',F.length(F.col('text')))
df_2.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



### Show the new dataframe

In [ ]:
df_2.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [ ]:
df_3 = df_2.groupBy('class').agg(F.avg(F.col('length')).alias('Avg_Length'))
df_3.show()

+-----+-----------------+
|class|       Avg_Length|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, CountVectorizer, StringIndexer, VectorAssembler

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="text_tok")

In [ ]:
stopWordsRemover = StopWordsRemover(inputCol="text_tok", outputCol="text_stopword")

In [ ]:
countVectorizer = CountVectorizer(inputCol="text_stopword", outputCol="text_cv")

In [ ]:
tf_idf = IDF(inputCol="text_cv", outputCol="text_tf_idf")

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

In [ ]:
#Convert the class column to index using StringIndexer
indexer = StringIndexer(inputCol="class", outputCol="classIndex",handleInvalid='skip') 

In [ ]:
vecAssembler = VectorAssembler(inputCols=['text_tf_idf','length'],outputCol='features')

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [ ]:
naiveclassifier = NaiveBayes(featuresCol='features',labelCol='classIndex',predictionCol='prediction')

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [ ]:
stages = [tokenizer, stopWordsRemover, countVectorizer, tf_idf, indexer, vecAssembler, naiveclassifier]

In [ ]:
from pyspark.ml import Pipeline
pl = Pipeline(stages=stages)

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

In [ ]:
trainDF, testDF = df_2.randomSplit([0.7,0.3],seed=42)

### Fit your Pipeline model to the training data

In [ ]:
pl_Model = pl.fit(trainDF)

### Perform predictions on tests dataframe

In [ ]:
predDF = pl_Model.transform(testDF)

### Print the schema of the prediction dataframe

In [ ]:
predDF.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- text_tok: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_stopword: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- text_cv: vector (nullable = true)
 |-- text_tf_idf: vector (nullable = true)
 |-- classIndex: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [ ]:
# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(labelCol="classIndex", predictionCol="prediction",
                                              metricName='fMeasureByLabel')
f_1 = evaluator.evaluate(predDF)
print("Test set accuracy = " + str(f_1))

Test set accuracy = 0.9839883551673945


In [ ]:
# from sklearn.metrics import f1_score
# print("sklearn F1 Score evaluator : ", f1_score(predDF.select('classIndex').toPandas(), predDF.select('prediction').toPandas()))

f1_score is: 0.9664707489549014


# GOOD LUCK
